# 第4章：使用CuPy进行科学计算

<img src="images/chapter-04/cupy_title.png" style="width:600px;"/>

CuPy是一个与NumPy和SciPy兼容的数组库，用于Python的GPU加速计算。CuPy可以作为替代品，在NVIDIA CUDA或AMD ROCm平台上运行现有的NumPy和SciPy代码。

CuPy是Chainer项目的一部分，但有来自包括NVIDIA在内的多个组织的维护者。CuPy实现了熟悉的NumPy API，但后端是用CUDA C++编写的。这使得已经熟悉NumPy的开发者只需切换导入语句，就能快速获得GPU加速。

## CuPy基础

CuPy是一个与NumPy/SciPy兼容的数组库，用于Python的GPU加速计算。CuPy可以作为替代品，在NVIDIA CUDA或AMD ROCm平台上运行现有的NumPy/SciPy代码。

CuPy为GPU设备提供多维数组、稀疏矩阵以及相关例程，所有这些都具有与NumPy和SciPy相同的API。

CuPy项目的目标是为Python用户提供GPU加速功能，而无需深入了解底层GPU技术。CuPy团队专注于提供：
- 完整的NumPy和SciPy API覆盖，成为完全的替代品，以及先进的CUDA功能以最大化性能。
- 成熟和高质量的库，作为所有需要加速的项目的基础包，从实验室环境到大规模集群。

### N维数组 / Cupy.ndarray数据结构

`cupy.ndarray`是NumPy `numpy.ndarray`的CuPy对应物。它为驻留在CUDA设备中的固定大小多维数组提供了直观的接口。

这个类实现了numpy.ndarray方法的子集。区别在于这个类在当前GPU设备上分配数组内容。

### 内存管理

CuPy默认使用内存池进行内存分配。内存池通过减少内存分配和CPU/GPU同步的开销显著提高了性能。

CuPy中有两种不同的内存池：
- 设备(GPU)内存池 - 用于GPU内存分配。
- 固定(CPU)内存池 - 在CPU到GPU数据传输期间使用的非可交换内存。

在大多数情况下，CuPy用户不需要了解内存分配和释放的细节，但了解CuPy中的这种优化对于对应用程序性能进行基准测试很重要。由于内存池中的缓存，您可能不会看到内存完全释放。

CuPy提供了控制此内存的高级API以及CUDA内存管理函数的低级API。

### 当前设备

CuPy有当前设备的概念，这是数组的分配、操作、计算等发生的默认GPU设备(默认id=0)。所有CuPy操作(除了多GPU功能和设备间复制)都在当前活动设备上执行。

通常，CuPy函数期望数组与当前设备在同一设备上。根据硬件配置，传递存储在非当前设备上的数组可能会工作，但通常不建议这样做，因为这可能会影响性能。

## API

### Cupy.ndarray

`cupy.ndarray`是CuPy生态系统的骨干，为`numpy.ndarray`提供了直观的对应物。`cupy.ndarray`与`numpy.ndarray`一样，是相同类型和大小的项目的固定大小多维容器。

### Cupy.ufuncs

在NumPy中，通用函数(简称ufunc)被定义为以逐元素方式对ndarray进行操作的函数，支持数组广播、类型转换和其他几个标准特性。换句话说，ufunc是一个'向量化'的包装器，用于接受固定数量的特定输入并产生固定数量的特定输出的函数。这些函数主要在NumPy数组上操作，构成了加速Python代码最强大的方式之一。(参见NumPy通用函数：https://numpy.org/doc/stable/reference/ufuncs.html)

类似地，CuPy实现了类似的ufunc，也支持广播、类型转换和输出类型确定。用户可以定义在`cupy.ndarray`对象上模仿NumPy ufuncs的`cupy.ufuncs`。

### NumPy和SciPy覆盖范围

可用的NumPy例程：https://docs.cupy.dev/en/stable/reference/routines.html 
可用的SciPy例程：https://docs.cupy.dev/en/stable/reference/scipy.html 

虽然CuPy设计为模仿NumPy，但使用CuPy有一些限制：
- 并非所有NumPy和SciPy函数都与CuPy兼容。
- CuPy可能并不总是提供显著的性能改进。
- 性能高度依赖于执行的操作和使用的硬件。

CuPy和NumPy之间还有一些差异，可能需要调整您的代码：
- 从浮点到整数的转换行为可能依赖于硬件。这是由于C++中类型转换的限制。
- 随机函数差异。NumPy的`random()`函数不支持`dtype`参数，但CuPy底层的随机数生成器cuRAND支持。
- 使用整数数组索引时，CuPy默认处理越界索引的方式与NumPy不同。NumPy会引发错误，但CuPy会环绕处理。
- 矩阵类型(`numpy.matrix`) - 当从稀疏矩阵计算密集矩阵时(例如，`coo_matrix + ndarray`)，SciPy返回`numpy.matrix`(`numpy.ndarray`的子类)。但是，CuPy对于此类操作返回`cupy.ndarray`。
- CuPy数组不能是非数值类型，如字符串或对象。
- CuPy中的通用函数只能与CuPy数组或标量一起使用。它们不接受其他对象(例如，列表或`numpy.ndarray`)。
- 与Numpy一样，CuPy的RandomState对象接受数字或完整numpy数组作为种子。
- NumPy的归约函数(例如`numpy.sum()`)返回标量值(例如`numpy.float32`)。但是CuPy对应函数返回零维的`cupy.ndarray`。

还有更多差异，但这些是最常遇到的。

## 编码指南

### 安装

在设置CuPy编程环境之前，首先确保您满足以下先决条件：
- CUDA兼容的GPU。(参见https://developer.nvidia.com/cuda-gpus 获取NVIDIA GPU列表)
- CUDA兼容的NVIDIA驱动程序。
- CUDA工具包

CUDA工具包的版本将决定您需要安装的NVIDIA驱动程序版本。CUDA工具包与包括Windows、Linux和macOS在内的许多操作系统兼容，但根据您打算使用的CUDA工具包版本，可能需要更新操作系统版本。

查看当前安装说明：https://docs.cupy.dev/en/stable/install.html

### 最佳实践

在将程序转换为CuPy之前，请确保使用NumPy和SciPy优化其在CPU上的实现。对初始实现进行基准测试将帮助您确定在移动到GPU时是否加速了程序。

要将处理从NumPy移动到CuPy，您需要：
- 导入CuPy。
- 将所有NumPy调用移动到CuPy。
  - CuPy覆盖了大部分NumPy API，所以先尝试这个。
- 将NumPy ndarray移动到CuPy ndarray
  - 使用`cupy.array()`或`cupy.asarray()`
- GPU处理后将CuPy ndarray转换回NumPy ndarray
  - 使用`cupy.asnumpy()`或`cupy.ndarray.get()`

例如，这个NumPy调用：

```python
import numpy as np
x_cpu = np.ones((1000,500,500))
```

对应这个CuPy调用：
```python
import cupy as cp
x_gpu = cp.ones((1000,500,500))
x_cpu = cp.asnumpy(x_gpu)
```

如果您正在对代码进行基准测试，需要显式调用`cp.cuda.Stream.null.synchronize()`以获得公平的计时。默认情况下，CuPy将并发运行GPU代码，函数将在GPU完成之前退出。调用`synchronize()`使我们等待GPU完成后再返回。

### 超越NumPy和SciPy

不幸的是，NumPy和SciPy不一定提供开发软件所需的所有功能。在这种情况下，您需要了解CuPy中的一些重要模式：

#### CuPy内核编译

CuPy提供了三个内核编译类。这个类的实例定义了一个CUDA内核，可以通过该实例的`__call__`方法调用：
- ElementwiseKernel - 像for循环一样在数组的每个元素上执行。
- ReductionKernel - 执行映射、归约和后归约函数。
- RawKernel - 使用原始CUDA源代码定义内核，可以控制网格大小、块大小等。

这些类型也可以使用`@cupyx.jit.*`装饰器对应物定义：`@cupyx.jit.elementwisekernel`、`@cupyx.jit.reductionkernel`和`@cupy.jit.rawkernel`。

#### CuPy类型通用内核

如果内核函数中的类型信息用一个字符定义，则被视为类型占位符。在整个函数中重复的相同字符将被推断为相同类型。这允许创建可重用的通用内核。

#### 在GPU设备之间移动

如果需要在GPU之间移动数据(从一个设备到另一个设备)，使用with语句创建上下文。如果您想在系统中的集成显卡和专用显卡之间切换，以满足能耗或性能考虑，可能需要这样做。

```python
import cupy as cp

device_id = 1

#为设备1创建上下文
with cp.cuda.Device(device_id):
   array_on_device1 = cp.array([1, 2, 3, 4, 5])

#超出上下文范围并在设备0上执行
array_on_device0 = cp.array([1, 2, 3, 4, 5]) 
```

### 性能考虑

#### 将数据从CPU移动到GPU

为了利用GPU，我们需要通过主板上的PCI总线将数据移动到GPU。这意味着我们需要将数据和代码移动到设备上以执行该代码。这样，CPU和GPU之间的PCI总线可能成为瓶颈。

将数据从CPU移动到GPU或反之都会产生一次性的性能成本。

#### 分支

具有许多逻辑分支的程序需要CPU。在CPU和GPU之间切换会产生可能影响性能的成本。根据在两个处理器之间切换的开销，具有大量if-then语句的程序可能更适合在CPU上运行。

确保您的函数是向量化的，以最小化分支。

#### 编译内核函数

当需要内核调用时，CuPy会编译针对给定参数的维度和dtypes优化的内核代码，将它们发送到GPU设备，并执行内核。然后CuPy会在进程中缓存发送到GPU设备的内核代码，这减少了后续调用的内核编译时间。

编译内核函数会产生一次性的性能成本。

#### 从当前设备移动数据

通常，CuPy函数期望数组与当前设备在同一设备上。类似于在CPU和GPU之间传递数据，根据硬件配置，传递存储在非当前设备上的数组可能会对性能产生负面影响。

当数据从一个设备移动到另一个设备时，会有性能权衡。

## 有用的参考链接
CuPy用户指南获取更多信息：https://docs.cupy.dev/en/stable/user_guide/index.html 

CuPy API参考：https://docs.cupy.dev/en/stable/reference/index.html 

CuPy Github仓库(包含更多示例)：https://github.com/cupy/cupy 

NumPy用户指南：https://numpy.org/doc/stable/user/ 

NumPy API指南：https://numpy.org/doc/stable/reference/index.html

# 示例

## 从NumPy到CuPy的简单转换

In [ ]:
import numpy as np
x_cpu = np.ones((1000,500,500))

x_cpu

In [ ]:
import cupy as cp
x_gpu = cp.ones((1000,500,500))
x_cpu = cp.asnumpy(x_gpu)

x_cpu

## 从NumPy到CuPy的更复杂转换

In [ ]:
import numpy as np

x_cpu = np.random.random((1000, 1000))
x_cpu *= 2 
u, s, v = np.linalg.svd(x_cpu)

u, s, v

In [ ]:
import cupy as cp

x_gpu = cp.random.random((1000, 1000))
x_gpu *= 2 
u, s, v = cp.linalg.svd(x_gpu)

u, s, v

## 添加用户定义的内核函数

In [ ]:
import cupy
from cupyx import jit


@jit.rawkernel()
def elementwise_copy(x, y, size):
    tid = jit.blockIdx.x * jit.blockDim.x + jit.threadIdx.x
    ntid = jit.gridDim.x * jit.blockDim.x
    for i in range(tid, size, ntid):
        y[i] = x[i]


size = cupy.uint32(2 ** 22)
x = cupy.random.normal(size=(size,), dtype=cupy.float32)
y = cupy.empty((size,), dtype=cupy.float32)

elementwise_copy((128,), (1024,), (x, y, size))

elementwise_copy[128, 1024](x, y, size)

assert (x == y).all()